In [1]:
import numpy as np
import pandas as pd

import ipywidgets as widgets
from IPython.display import display, Math, clear_output

import queueing
import ctmc

Loading BokehJS ...

In [2]:
import bokeh.plotting as bplt
from bokeh.models import Range1d, LabelSet, ColumnDataSource
from bokeh.models import Arrow, TeeHead, VeeHead, NormalHead, OpenHead
from bokeh.models import Whisker, Span
from bokeh.models.glyphs import Step
from bokeh.models.markers import Circle
from bokeh.io import output_notebook, show, push_notebook
output_notebook()

Loading BokehJS ...

# Queueing Theory: Section 1 - Numerical Example

Suppose you are operating a lab-testing facility (e.g., at a hospital) that receives samples and processes them.  Assume that samples arrive according to a Poisson process with rate $\lambda = 12$ samples per hour and each sample requires a processing time that is random, follows an $\text{Exponential}(\mu)$ distribution, and is independent of everything else.  Let $\mu = 15$ per hour (so the average processing time is $4$ minutes: $1/\mu = 1/15$ hours).

The lab can only process one sample at a time and any samples that arrive while one is currently being processed waits in a queue.  As soon as a sample is finished, the next sample from the queue starts (where the next sample is determined by a “first-come first-served“ policy).  If there are no samples in the queue, then the lab remains idle until the next sample arrives.

Assume that samples can wait indefinitely and there is unlimited space to store samples until they are processed.  Assume also that at time $0$, the lab is idle and awaiting the first sample.

## Simulated Sample Path

In the below code, we simulate a sample path of this system.  Note that the dynamics of the system are a result of two stochastic “primitives”:
* arrival times of customers (Poisson process with rate $\lambda$)
* service requirements of customers (exponentially distributed with rate $\mu$)

From these stochastic primitives, we can determine various other metrics such as the number of samples waiting at any time, how long a sample waits, etc.  

In the table below, we calculate the “wait time” for each sample, which is the amount of time that elapses between when a sample arrives and when it *begins* processing, essentially the amount of time spent “in line.”  The departure time is simply the arrival time, plus the wait time, plus the service requirement (processing time).

In [3]:
lam = 8 # arrival rate
mu = 10 # service requirement parameter
T = 100 # time horizon

num_arr = np.random.poisson(lam*T)
arr_times = np.sort(T*np.random.rand(num_arr))
svc_reqs = np.random.exponential(1/mu, num_arr)
wait_times = queueing.lindley_recursion(svc_reqs[:-1] - np.diff(arr_times))
dep_times = arr_times + wait_times + svc_reqs
A = pd.DataFrame(
    { 'arrival time': arr_times, 
      'service req': svc_reqs,
      'wait time': wait_times,
      'departure time': dep_times,
    },
    index=pd.Index(range(1,num_arr+1)))

widgets.interact(lambda customer: queueing.sliding_table(A, customer-1), customer=(1, len(A.index)-1));

interactive(children=(IntSlider(value=427, description='customer', max=854, min=1), Output()), _dom_classes=('…

The particular sequence of arrivals and departures can also be used to determine the number of samples in the lab (either waiting or being processed) over time.  Define the stochastic process $\{X(t),\, t \geq 0\}$ to be the “headcount process” where $X(t) = i$ means that there are $i$ samples in the lab at time $t$.  (Note that, if $i \geq 1$ then one of those samples is in being processed and the remaining $i-1$, if any, are waiting for processing; if $i = 0$ then the lab is idle.)

The headcount process sample path is shown below.  Note that it increases by 1 when an arrival occurs and decreases by 1 when a departure occurs.

In [4]:
sample_path = queueing.headcount_path(arr_times, dep_times, T)

In [5]:
dt = 0.05
t_slider = widgets.FloatSlider(value=0, 
                               min=min(sample_path['start']),
                               max=max(sample_path['end']), 
                               step=dt,
                               description='t')
display(t_slider)
p, h, t_mark = ctmc.plot_path(sample_path, list(set(sample_path['X'].values)))
widgets.interactive_output(lambda t: ctmc.path_show_t(t, sample_path, p, h, t_mark), {'t': t_slider});

FloatSlider(value=0.0, description='t', step=0.05)